In [172]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [173]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train.set_index('PassengerId',inplace=True)
train.head()

In [174]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.set_index('PassengerId',inplace=True)
test.head()

In [175]:
train.columns

**Now i'll give real integers to each different value in the 'Embarked' and the 'Sex' columns**

**Here i gave the 2 entries with 'nan' values a value 0 in the 'Embarked' columns**

In [176]:
train['Embarked']=train['Embarked'].apply(lambda i:str(i))
print('Number of nan entries:',len(train[train['Embarked']=='nan']))
train['Embarked'].replace({'S':1,'Q':2,'C':3,'nan':0},inplace=True)
train['Sex']=train['Sex'].replace({'female':0,'male':1})


**We have here the 'Cabin' column where the entries are the name of the cabin, but there is ones with 'nan' values so i replaced it with 'NoCabin' and ill do onehotencoding using the sklearn onehotencoder and apply it also on the test set.**

In [177]:


train['Cabin']=train['Cabin'].apply(lambda x:str(x))
train['Cabin'].replace({'nan':'NoCabin'},inplace=True)
from sklearn.preprocessing import OneHotEncoder

ohe1=OneHotEncoder(categories='auto',  drop=None, sparse=True, handle_unknown='ignore' )      
transformed = ohe1.fit_transform(train[['Cabin']])
x=pd.DataFrame(transformed.toarray())
x.columns=ohe1.categories_
x.index=train.index
train=train.join(x).drop(['Cabin'],axis=1)

**Now i'll train a second one hot encoder on the 'Ticket' column**

In [178]:

from sklearn.preprocessing import OneHotEncoder

ohe2=OneHotEncoder(categories='auto',  drop=None, sparse=True, handle_unknown='ignore' )      
transformed = ohe2.fit_transform(train[['Ticket']])
x=pd.DataFrame(transformed.toarray())
x.columns=ohe2.categories_
x.index=train.index
train=train.join(x).drop(['Ticket'],axis=1)

**The name won't affect the result i guess and it's unique for every person so it should not be corrolated to the 'Survived' column, so i dropped it**

In [179]:

train.drop(['Name'],axis=1,inplace=True)

**In the 'Age'column ,there is some missing values so i replaced it with the mean value of the column**

In [180]:

import math
mean_value=math.floor(np.mean(train['Age']))+1
train['Age'].fillna(value=mean_value, inplace=True)


In [181]:
train

In [182]:
y_train=train['Survived']
X_train=train.drop(['Survived'],axis=1)

**Now i'll do the same preprocessing to the 'test' set as if they are coming from the same dataset as the 'train' dataset so the same mistakes.**

**And ill apply the one hot encoders trained on the 'train' dataset on the test dataset**

In [183]:
test['Fare'].fillna(value=np.mean(test['Fare']),inplace=True)
test['Embarked']=test['Embarked'].apply(lambda i:str(i))
print('Number of nan entries:',len(test[test['Embarked']=='nan']))
test['Embarked'].replace({'S':1,'Q':2,'C':3,'nan':0},inplace=True)
test['Sex']=test['Sex'].replace({'female':0,'male':1})
test['Cabin']=test['Cabin'].apply(lambda x:str(x))
test['Cabin'].replace({'nan':'NoCabin'},inplace=True)
transformed = ohe1.transform(test[['Cabin']])
x=pd.DataFrame(transformed.toarray())
x.columns=ohe1.categories_
x.index=test.index
test=test.join(x).drop(['Cabin'],axis=1)
transformed = ohe2.transform(test[['Ticket']])
x=pd.DataFrame(transformed.toarray())
x.columns=ohe2.categories_
x.index=test.index
test=test.join(x).drop(['Ticket'],axis=1)
test.drop(['Name'],axis=1,inplace=True)
import math
mean_value=math.floor(np.mean(test['Age']))+1
test['Age'].fillna(value=mean_value, inplace=True)
test


In [191]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf2 = GradientBoostingClassifier()
gb_clf2.fit(X_train, y_train)
predictions = gb_clf2.predict(test)


In [213]:
output=pd.DataFrame(predictions)
output.columns=['Survived']
output.set_index(test.index,inplace=True)
output

In [214]:
output.to_csv('CharbelTitanicProject')